In [32]:
import pandas as pd
from pathlib import Path
import re
import numpy as np

In [12]:
example_path = Path('/gws/nopw/j04/iecdt/cheetah/2017_08_29/bottom/phantom/flick2')

In [85]:
df = pd.read_hdf(example_path / "filtered_2D" / "cam3DLC_resnet152_CheetahOct14shuffle1_500000.h5")

In [87]:
display(df)

scorer    DLC_resnet152_CheetahOct14shuffle1_500000                         \
bodyparts                                     r_eye                          
coords                                            x           y likelihood   
0                                        466.780748  496.310020   0.843646   
1                                        476.662960  496.780764   0.867980   
2                                        467.291212  496.760568   0.816340   
3                                        467.653452  496.879998   0.833451   
4                                        476.905672  496.717315   0.845231   
..                                              ...         ...        ...   
316                                             NaN         NaN   0.031859   
317                                             NaN         NaN   0.040030   
318                                             NaN         NaN   0.010316   
319                                             NaN         NaN   0.173078   
320                                             NaN         NaN   0.221967   

scorer                                                                \
bodyparts       l_eye                         r_shoulder               
coords              x           y likelihood           x           y   
0          477.761635  497.109234   0.865188  468.364375  505.838816   
1          477.798796  497.308653   0.812952  467.206840  513.339016   
2          477.738068  497.299124   0.826227  467.445744  513.202022   
3          477.482832  497.376827   0.834253  467.456618  505.628165   
4          477.585308  497.492764   0.901569  478.050518  513.924471   
..                ...         ...        ...         ...         ...   
316               NaN         NaN   0.081492  477.285982  496.974659   
317               NaN         NaN   0.096777  478.092237  505.751912   
318               NaN         NaN   0.012763  477.125541  497.005572   
319               NaN         NaN   0.028792         NaN         NaN   
320               NaN         NaN   0.170158         NaN         NaN   

scorer                             ...                                     \
bodyparts            r_front_knee  ...       lure   tail_base               
coords    likelihood            x  ... likelihood           x           y   
0           0.648893   477.975610  ...   0.099997  458.291532  496.340185   
1           0.706509   467.915626  ...   0.048126  458.457967  496.305668   
2           0.631165   467.901369  ...   0.060170  458.221192  496.353792   
3           0.480500   467.914589  ...   0.075540  467.733226  496.262832   
4           0.476722   467.821035  ...   0.070295  467.384385  496.280282   
..               ...          ...  ...        ...         ...         ...   
316         0.590266          NaN  ...   0.056302  467.267407  496.250117   
317         0.430013          NaN  ...   0.049474  467.066560  496.618535   
318         0.340691          NaN  ...   0.023850  457.543912  496.777785   
319         0.163894          NaN  ...   0.040336  466.571745  496.823254   
320         0.136387          NaN  ...   0.032044  466.655539  496.822721   

scorer                                                               \
bodyparts                   nose                          neck_base   
coords    likelihood           x           y likelihood           x   
0           0.779278  467.133634  504.481726   0.874825  477.818254   
1           0.708053  478.029599  505.899045   0.845803         NaN   
2           0.559946  467.599322  505.047461   0.829336         NaN   
3           0.499875  467.399360  505.536394   0.850702         NaN   
4           0.394748  477.986238  506.087321   0.885310         NaN   
..               ...         ...         ...        ...         ...   
316         0.851325         NaN         NaN   0.094053  477.544277   
317         0.901719         NaN         NaN   0.089918         NaN   
318         0.847547         NaN      

In [ ]:
class Sequence:
    def __init__(self, path: Path):
        self.path = path
        self.filtered_dir = self.path / "filtered_2D"
        self.samples = [] # TODO: this will be a list of sample objects. each sample object represents a frame of a specific video from a specific camera. 
        # anything else we need to store?
    
    def generate_samples(self):

        # this will return a list(or other datastructure?) of Sample objects
        # first we need to load the dataframe for each camera file

        for file in self.filtered_dir.glob("cam*.h5"):
            match = re.search(r'cam(\d+)', file.name)
            if not match:
                print(f"Warning: Could not parse camera ID from {file.name}")
                continue

            cam_key = match.group(1)  # e.g. "3"
            print(f"Loading filtered detections from {file} ({cam_key})")

            df = pd.read_hdf(file, key="df_with_missing")

            #TODO clean this up
            # drop the uppermost level because we don't need it
            new_df = df.copy()
            new_df.columns = new_df.columns.droplevel(0)

            # go through the frames
            num_frames = len(new_df)

            bodyparts = new_df.columns.get_level_values("bodyparts")
            num_bodyparts = len(bodyparts)

            # we know shape of tensor (C, J, 3)
            for i in range(num_frames):
                tensor = np.zeros((6, num_bodyparts, 3), dtype=np.float32)

                for body_idx, bodypart in enumerate(bodyparts):
                    u = new_df[(bodypart, 'x')].iloc[i]
                    v = new_df[(bodypart, 'y')].iloc[i]
                    c = new_df[(bodypart, 'likelihood')].iloc[i]

                    tensor[int(cam_key)-1, body_idx] = [u,v,c]

                    # TODO: this should be returning a populated Sample object 
                    # we will want to store the frame id as part of this, as well as the filepath of where it came from

                    self.samples.append(tensor)



In [92]:
test = Sequence(example_path)

In [93]:
test.generate_samples()

Loading filtered detections from /gws/nopw/j04/iecdt/cheetah/2017_08_29/bottom/phantom/flick2/filtered_2D/cam3DLC_resnet152_CheetahOct14shuffle1_500000.h5 (3)
Loading filtered detections from /gws/nopw/j04/iecdt/cheetah/2017_08_29/bottom/phantom/flick2/filtered_2D/cam4DLC_resnet152_CheetahOct14shuffle1_500000.h5 (4)
Loading filtered detections from /gws/nopw/j04/iecdt/cheetah/2017_08_29/bottom/phantom/flick2/filtered_2D/cam5DLC_resnet152_CheetahOct14shuffle1_500000.h5 (5)
Loading filtered detections from /gws/nopw/j04/iecdt/cheetah/2017_08_29/bottom/phantom/flick2/filtered_2D/cam6DLC_resnet152_CheetahOct14shuffle1_500000.h5 (6)


In [94]:
frame = test.samples[2]

In [95]:
frame[2]

array([[4.6678076e+02, 4.9631003e+02, 8.4364593e-01],
       [4.6678076e+02, 4.9631003e+02, 8.4364593e-01],
       [4.6678076e+02, 4.9631003e+02, 8.4364593e-01],
       [4.7776163e+02, 4.9710922e+02, 8.6518836e-01],
       [4.7776163e+02, 4.9710922e+02, 8.6518836e-01],
       [4.7776163e+02, 4.9710922e+02, 8.6518836e-01],
       [4.6836438e+02, 5.0583881e+02, 6.4889258e-01],
       [4.6836438e+02, 5.0583881e+02, 6.4889258e-01],
       [4.6836438e+02, 5.0583881e+02, 6.4889258e-01],
       [4.7797562e+02, 5.2255011e+02, 7.2689044e-01],
       [4.7797562e+02, 5.2255011e+02, 7.2689044e-01],
       [4.7797562e+02, 5.2255011e+02, 7.2689044e-01],
       [4.7764520e+02, 5.2241718e+02, 6.5971571e-01],
       [4.7764520e+02, 5.2241718e+02, 6.5971571e-01],
       [4.7764520e+02, 5.2241718e+02, 6.5971571e-01],
       [4.7782736e+02, 5.2233105e+02, 4.8096856e-01],
       [4.7782736e+02, 5.2233105e+02, 4.8096856e-01],
       [4.7782736e+02, 5.2233105e+02, 4.8096856e-01],
       [4.6786768e+02, 4.978